In [1]:
from rich.console import Console
from rich.live import Live
from rich.text import Text
import time

console = Console()

# Create a function to display the animation
def animate_running():
    with Live(refresh_per_second=4) as live:
        for i in range(40):  # Run the loop for 10 seconds (4 frames per second)
            num_dots = (i % 3) + 1  # Cycle through 1, 2, 3 dots
            text = Text(f"Running{'.' * num_dots}", style="blue")
            live.update(text)
            time.sleep(0.25)  # Update every 0.25 seconds

# Call the function
animate_running()

Running.

In [34]:
import sys
sys.path.append('/Users/varantzanoyan/repos/chronon/api/py')
%env PYTHONPATH=/Users/varantzanoyan/repos/chronon/airflow/zipline:/Users/varantzanoyan/repos/chronon/api/py
%load_ext autoreload
%autoreload 2
from group_bys.sample_team.transactions import v1
from joins.sample_team.chargeback import v1 as chargeback_features

env: PYTHONPATH=/Users/varantzanoyan/repos/chronon/airflow/zipline:/Users/varantzanoyan/repos/chronon/api/py
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
from joins.sample_team.chargeback import v1 as chargeback_features
chargeback_features.run()

Running: None
├── compute left (SUCCEEDED) ]8;id=40607;http://localhost:8000/compute_left.log\compute_left.log]8;;\
│   └── join parts
│       └── compute sample_team.transactions.v1 (SUCCEEDED) ]8;id=11597;http://localhost:8000/compute_sample_team.transactions.v1.log\compute_sample_team.transactions.v1.log]8;;\
└── compute final (SUCCEEDED) ]8;id=429114;http://localhost:8000/compute_final.log\compute_final.log]8;;\

In [1]:
import time
import threading
from pathlib import Path
from rich.console import Console
from rich.tree import Tree
from rich.live import Live
from rich.markup import escape

console = Console()

class Task:
    def __init__(self, name, command):
        self.name = name
        self.command = command
        self.status = "WAITING"
        self.done = False
        self.animation_step = 0
        self.log_file = Path.home() / "logs" / f"{name.replace(' ', '_')}.log"

    def run(self):
        self.status = "RUNNING"
        self.execute_command()
        self.status = "SUCCEEDED"
        self.done = True

    def execute_command(self):
        print(f"Executing: {self.command}")
        time.sleep(2)  # Simulate task execution

def get_status_display(task):
    color = {
        "WAITING": "yellow",
        "RUNNING": "blue",
        "SUCCEEDED": "green"
    }.get(task.status, "white")

    if task.status == "RUNNING":
        dots = "." * ((task.animation_step % 3) + 1)
        task.animation_step += 1
        status_text = f"[{color}]RUNNING{dots: <3}[/{color}]"
    else:
        status_text = f"[{color}]{task.status}[/{color}]"
    
    log_path = task.log_file.resolve()
    file_url = f"file://{log_path}"
    link_text = f"[link={file_url}]{escape(str(log_path))}[/link]"
    return f"{task.name} ({status_text}) {link_text}"

def create_dag_tree(tasks):
    tree = Tree("Join DAG Visualization")
    left_branch = tree.add(get_status_display(tasks[0]))
    parts_branch = left_branch.add("join parts")
    for task in tasks[1:-1]:
        parts_branch.add(get_status_display(task))
    tree.add(get_status_display(tasks[-1]))
    return tree

def run_task(task):
    task.run()

def animate_tasks(live, tasks):
    while any(task.status == "RUNNING" for task in tasks):
        for task in tasks:
            if task.status == "RUNNING":
                task.animation_step += 1
        live.update(create_dag_tree(tasks))
        time.sleep(0.25)

def run(join):
    tasks = [
        Task("compute left", "echo 'computing left...'"),
        *[Task(f"compute {part.groupBy.metaData.name}", f"echo 'computing {part.groupBy.metaData.name}...'") for part in join.joinParts],
        Task("compute final", "echo 'computing final...'")
    ]
    
    with Live(create_dag_tree(tasks), refresh_per_second=4) as live:
        # Run compute left
        tasks[0].status = "RUNNING"
        left_thread = threading.Thread(target=run_task, args=(tasks[0],))
        left_thread.start()
        animate_tasks(live, tasks)
        left_thread.join()
        
        # Run join parts in parallel
        for task in tasks[1:-1]:
            task.status = "RUNNING"
        threads = [threading.Thread(target=run_task, args=(task,)) for task in tasks[1:-1]]
        for thread in threads:
            thread.start()
        animate_tasks(live, tasks)
        for thread in threads:
            thread.join()
        
        # Run compute final
        tasks[-1].status = "RUNNING"
        final_thread = threading.Thread(target=run_task, args=(tasks[-1],))
        final_thread.start()
        animate_tasks(live, tasks)
        final_thread.join()
        
        # Ensure final state is displayed
        live.update(create_dag_tree(tasks))
        
        # Add a small delay to ensure the final state is visible
        time.sleep(0.5)

# Example usage
if __name__ == "__main__":
    run(chargeback_features)

NameError: name 'chargeback_features' is not defined

In [22]:
! echo "file:////Users/varantzanoyan/logs/compute_left.log"

file:////Users/varantzanoyan/logs/compute_left.log


In [24]:
print( "<a href=\"file:///{log_path}\">Link 1</a>")

<a href="file:///{log_path}">Link 1</a>


In [33]:
from IPython.display import FileLink

# Create a FileLink object pointing to the desired file
file_link = FileLink("teams.json")

print(file_link)

# Display the link in the output
x = display(file_link)
print(x)

/Users/varantzanoyan/repos/chronon/airflow/zipline/teams.json


/Users/varantzanoyan/repos/chronon/airflow/zipline/teams.json

None
